In [6]:
!pip install librosa 
import librosa

In [0]:
from google.colab import files

uploaded = files.upload()

Saving trn0000.wav to trn0000.wav
Saving trn0001.wav to trn0001.wav
Saving trn0002.wav to trn0002.wav
Saving trn0003.wav to trn0003.wav
Saving trn0004.wav to trn0004.wav
Saving trn0005.wav to trn0005.wav
Saving trn0006.wav to trn0006.wav
Saving trn0007.wav to trn0007.wav
Saving trn0008.wav to trn0008.wav
Saving trn0009.wav to trn0009.wav
Saving trn0010.wav to trn0010.wav
Saving trn0011.wav to trn0011.wav
Saving trn0012.wav to trn0012.wav
Saving trn0013.wav to trn0013.wav
Saving trn0014.wav to trn0014.wav
Saving trn0015.wav to trn0015.wav
Saving trn0016.wav to trn0016.wav
Saving trn0017.wav to trn0017.wav
Saving trn0018.wav to trn0018.wav
Saving trn0019.wav to trn0019.wav
Saving trn0020.wav to trn0020.wav
Saving trn0021.wav to trn0021.wav
Saving trn0022.wav to trn0022.wav
Saving trn0023.wav to trn0023.wav
Saving trn0024.wav to trn0024.wav
Saving trn0025.wav to trn0025.wav
Saving trn0026.wav to trn0026.wav
Saving trn0027.wav to trn0027.wav
Saving trn0028.wav to trn0028.wav
Saving trn0029

In [4]:
!ls

sample_data


**Read all the 3 sets of data and store them into an array/List.**

In [0]:
import glob
import numpy as np
S_cs_lst = []
T_S = []

for file in sorted(glob.iglob('/content/trs*.wav')):
    cs, sr=librosa.load(file, sr=None)
    S=librosa.stft(cs, n_fft=1024, hop_length=512)
    
    # Intermediate array which stores the Absolute and transposed spectogram data
    Inter_S = np.transpose(np.abs(S))
    #print(Inter_S.shape)
    
    # appending the actual length of signal
    T_S.append(Inter_S.shape[0])
    
    # Padded 0's and made all the spectograms of equal length. Selected shape is [300 , 513]
    result_S = np.zeros([300,513], dtype=np.float)
    result_S[:Inter_S.shape[0], :Inter_S.shape[1]] = Inter_S
    
    S_cs_lst.append(result_S)
    
S_cs_lst = np.array(S_cs_lst)



In [0]:
N_ns_lst = []
T_N =[]

for file in sorted(glob.iglob('/content/trn*.wav')):
    ns, n_sr=librosa.load(file, sr=None)
    N=librosa.stft(ns, n_fft=1024, hop_length=512)
    
    # Intermediate array which stores the Absolute and transposed spectogram data
    Inter_N = np.transpose(np.abs(N))
    #print(Inter_N.shape)
    
    # appending the actual length of signal
    T_N.append(Inter_N.shape[0])
    
    # Padded 0's and made all the spectograms of equal length. Selected shape is [300 , 513]
    result_N = np.zeros([300,513], dtype=np.float)
    result_N[:Inter_N.shape[0], :Inter_N.shape[1]] = Inter_N
    
    
    N_ns_lst.append(result_N)
N_ns_lst = np.array(N_ns_lst)




In [0]:

X_xs_lst = []
T_X=[]

for file in sorted(glob.iglob('/content/trx*.wav')):
    xs, x_sr=librosa.load(file, sr=None)
    X=librosa.stft(xs, n_fft=1024, hop_length=512)
    
    # Intermediate array which stores the Absolute and transposed spectogram data
    Inter_X = np.transpose(np.abs(X))
    #print(Inter_X.shape)
    
    # appending the actual length of signal
    T_X.append(Inter_X.shape[0])
    
    # Padded 0's and made all the spectograms of equal length. Selected shape is [300 , 513]
    result_X = np.zeros([300,513], dtype=np.float)
    result_X[:Inter_X.shape[0], :Inter_X.shape[1]] = Inter_X
    
    
    X_xs_lst.append(result_X)
X_xs_lst = np.array(X_xs_lst)


In [0]:
# Shape for the input data set

print(S_cs_lst.shape)
print(N_ns_lst.shape)
print(X_xs_lst.shape)

(1200, 300, 513)
(1200, 300, 513)
(1200, 300, 513)


In [0]:
# Create a M matrix by comparing the S and N matrix

M_ms_lst=[]

for i in range(1200):
  compare_lst = list((S_cs_lst[i] > N_ns_lst[i]).astype(np.int))
  #print("compare_lst.shape:",  np.array(compare_lst).shape)
  M_ms_lst.append(compare_lst)
print("M_ms_lst.shape:", np.array(M_ms_lst).shape)
  

M_ms_lst.shape: (1200, 300, 513)


In [0]:
# Multiply M and X matrix

import numpy as np
S_hat = np.multiply(np.array(M_ms_lst) , X_xs_lst)

In [0]:
S_hat.shape

(1200, 300, 513)

In [0]:
# Reset Grapphs

import tensorflow as tf
tf.reset_default_graph()


In [0]:
inputs=513
batch_size=10
length_of_files=len(X_xs_lst)
#num_time_steps=300
num_neurons=100
learning_rate=0.02
num_iteration=120
num_outputs=513


**Train RNN**

In [0]:
# retrain th emodel against S_hat
tf.reset_default_graph()

X_new = tf.placeholder('float',[None,None,inputs])
y_new = tf.placeholder('float',[None,None,num_outputs])

cell_new = tf.contrib.rnn.OutputProjectionWrapper(tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.BasicLSTMCell(num_units=num_neurons, activation=tf.nn.relu), output_keep_prob=0.8),output_size=num_outputs)

outputs_new, states_new = tf.nn.dynamic_rnn(cell_new, X_new, dtype=tf.float32)

loss_new = tf.reduce_mean(tf.square(outputs_new - y_new)) # MSE
optimizer_new = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_new = optimizer_new.minimize(loss_new)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
# batch Function

def next_batch_new(X_xs_lst,batch_size,T):
  for i in range(0, length_of_files, batch_size):
    #print("i:", i)
    batch_x = np.array([ X_xs_lst[i+j][:T[i+j],:] for j in range(batch_size)])
    #print("batch_x.shape intermediate:", batch_x.shape)
    
  return batch_x

# initialize session
session = tf.InteractiveSession()
initalize = tf.global_variables_initializer()
saver_new = tf.train.Saver()
final_outputs_new=[]
session.run(initalize)

for iteration in range(num_iteration):
  # Pass S_hat which was created in the last step as a np.multiple(M * S)
  X_batch_new = next_batch_new(X_xs_lst,batch_size,T_X)
  y_batch_new = next_batch_new(S_hat,batch_size,T_S)
  session.run(train_new, feed_dict={X_new: X_batch_new, y_new: y_batch_new})
  
  if (iteration % 10) == 0:
    mse_new = loss_new.eval(feed_dict={X_new: X_batch_new, y_new: y_batch_new})
    out_new = outputs_new.eval(feed_dict={X_new: X_batch_new, y_new: y_batch_new})
    final_outputs_new.append(list(out_new))
    print(iteration, "\tMSE:", mse_new, "\t", " and X_batch.shape:", X_batch_new.shape, "and y_batch.shape", y_batch_new.shape, "and out.shape:", out_new.shape, "and final_outputs.shape:", np.array(final_outputs_new).shape)

# Save Model for Later
saver_new.save(session, "./ex_audio_final_model_new")

0 	MSE: 0.28743684 	  and X_batch.shape: (10, 90, 513) and y_batch.shape (10, 90, 513) and out.shape: (10, 90, 513) and final_outputs.shape: (1, 10, 90, 513)
10 	MSE: 0.026204439 	  and X_batch.shape: (10, 90, 513) and y_batch.shape (10, 90, 513) and out.shape: (10, 90, 513) and final_outputs.shape: (2, 10, 90, 513)
20 	MSE: 0.021381088 	  and X_batch.shape: (10, 90, 513) and y_batch.shape (10, 90, 513) and out.shape: (10, 90, 513) and final_outputs.shape: (3, 10, 90, 513)
30 	MSE: 0.019660765 	  and X_batch.shape: (10, 90, 513) and y_batch.shape (10, 90, 513) and out.shape: (10, 90, 513) and final_outputs.shape: (4, 10, 90, 513)
40 	MSE: 0.02010632 	  and X_batch.shape: (10, 90, 513) and y_batch.shape (10, 90, 513) and out.shape: (10, 90, 513) and final_outputs.shape: (5, 10, 90, 513)
50 	MSE: 0.02010813 	  and X_batch.shape: (10, 90, 513) and y_batch.shape (10, 90, 513) and out.shape: (10, 90, 513) and final_outputs.shape: (6, 10, 90, 513)
60 	MSE: 0.019937985 	  and X_batch.shape: (

'./ex_audio_final_model_new'

**Changed the Loss function to see how it is impacting.**

In [0]:
tf.reset_default_graph()

In [0]:
inputs=513
batch_size=10
length_of_files=len(X_xs_lst)
#num_time_steps=300
num_neurons=100
learning_rate=0.03
num_iteration=120
num_outputs=513

# retrain the model against S_hat


X = tf.placeholder('float',[None,None,inputs])
y = tf.placeholder('float',[None,None,num_outputs])

cell = tf.contrib.rnn.OutputProjectionWrapper(tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.BasicLSTMCell(num_units=num_neurons, activation=tf.nn.relu), output_keep_prob=0.8),output_size=num_outputs)

outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

loss = tf.reduce_mean(tf.abs(outputs - y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train = optimizer_new.minimize(loss)


In [0]:
# Batch Function
 
def next_batch_new(X_xs_lst,batch_size,T):
  for i in range(0, length_of_files, batch_size):
    #print("i:", i)
    batch_x = np.array([ X_xs_lst[i+j][:T[i+j],:] for j in range(batch_size)])
    #print("batch_x.shape intermediate:", batch_x.shape)
    
  return batch_x

# initialize session
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
saver = tf.train.Saver()
final_outputs=[]

sess.run(init)

for iteration in range(num_iteration):
  # Pass S_hat which was created in the last step as a np.multiple(M * S)
  X_batch = next_batch_new(X_xs_lst,batch_size,T_X)
  y_batch = next_batch_new(S_hat,batch_size,T_S)
  
  sess.run(train, feed_dict={X: X_batch, y: y_batch})
  
  if (iteration % 10) == 0:
    abe = loss.eval(feed_dict={X: X_batch, y: y_batch}, session=sess)
    out = outputs.eval(feed_dict={X: X_batch, y: y_batch}, session=sess)
    final_outputs.append(list(out))
    print(iteration, "\tABE:", abe, "\t", " and X_batch.shape:", X_batch_new.shape, "and y_batch.shape", y_batch_new.shape, "and out.shape:", out_new.shape, "and final_outputs.shape:", np.array(final_outputs).shape)

# Save Model for Later
saver.save(sess, "./ex_audio_final_model_new_v1")  

    

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


0 	ABE: 3.633687 	  and X_batch.shape: (10, 90, 513) and y_batch.shape (10, 90, 513) and out.shape: (10, 90, 513) and final_outputs.shape: (1, 10, 90, 513)
10 	ABE: 0.043349545 	  and X_batch.shape: (10, 90, 513) and y_batch.shape (10, 90, 513) and out.shape: (10, 90, 513) and final_outputs.shape: (2, 10, 90, 513)
20 	ABE: 0.0390972 	  and X_batch.shape: (10, 90, 513) and y_batch.shape (10, 90, 513) and out.shape: (10, 90, 513) and final_outputs.shape: (3, 10, 90, 513)
30 	ABE: 0.03815271 	  and X_batch.shape: (10, 90, 513) and y_batch.shape (10, 90, 513) and out.shape: (10, 90, 513) and final_outputs.shape: (4, 10, 90, 513)
40 	ABE: 0.03772956 	  and X_batch.shape: (10, 90, 513) and y_batch.shape (10, 90, 513) and out.shape: (10, 90, 513) and final_outputs.shape: (5, 10, 90, 513)
50 	ABE: 0.037617322 	  and X_batch.shape: (10, 90, 513) and y_batch.shape (10, 90, 513) and out.shape: (10, 90, 513) and final_outputs.shape: (6, 10, 90, 513)
60 	ABE: 0.03774608 	  and X_batch.shape: (10, 9

'./ex_audio_final_model_new_v1'

In [0]:
!ls ex*

ex_audio_final_model_new.data-00000-of-00001
ex_audio_final_model_new.index
ex_audio_final_model_new.meta
ex_audio_final_model_new_v1.data-00000-of-00001
ex_audio_final_model_new_v1.index
ex_audio_final_model_new_v1.meta


**Uploading Validation dataset**

In [0]:
from google.colab import files

uploaded = files.upload()

Saving vs0021.wav to vs0021.wav
Saving vs0022.wav to vs0022.wav
Saving vs0023.wav to vs0023.wav
Saving vs0024.wav to vs0024.wav
Saving vs0025.wav to vs0025.wav
Saving vs0026.wav to vs0026.wav
Saving vs0027.wav to vs0027.wav
Saving vs0028.wav to vs0028.wav
Saving vs0029.wav to vs0029.wav
Saving vs0030.wav to vs0030.wav
Saving vs0031.wav to vs0031.wav
Saving vs0032.wav to vs0032.wav
Saving vs0033.wav to vs0033.wav
Saving vs0034.wav to vs0034.wav
Saving vs0035.wav to vs0035.wav
Saving vs0036.wav to vs0036.wav
Saving vs0037.wav to vs0037.wav
Saving vs0038.wav to vs0038.wav
Saving vs0039.wav to vs0039.wav
Saving vs0040.wav to vs0040.wav
Saving vs0041.wav to vs0041.wav
Saving vs0042.wav to vs0042.wav
Saving vs0043.wav to vs0043.wav
Saving vs0044.wav to vs0044.wav
Saving vs0045.wav to vs0045.wav
Saving vs0046.wav to vs0046.wav
Saving vs0047.wav to vs0047.wav
Saving vs0048.wav to vs0048.wav
Saving vs0049.wav to vs0049.wav
Saving vs0050.wav to vs0050.wav
Saving vs0051.wav to vs0051.wav
Saving v

In [0]:
!ls vx*|wc -l

1200


**Capturing data into Arrays. Here, while loading all the files the Colab session always crashed. Hence, at the end i used 100 files.**

In [0]:
# Capturing data in to arrays.

import glob
import numpy as np

S_cs_lst_Val = []
T_S_Val = []

vs_lst =[]
S_Val_lst=[]

for file in sorted(glob.iglob('/content/vs0*.wav')):
    vs, sr=librosa.load(file, sr=None)
    
    S_Val=librosa.stft(vs, n_fft=1024, hop_length=512)

    vs_lst.append(vs)

    
    # Intermediate array which stores the Absolute and transposed spectogram data
    Inter_S_Val = np.transpose(np.abs(S_Val))
    #print(Inter_S.shape)
    
    # appending the actual length of signal
    T_S_Val.append(Inter_S_Val.shape[0])

    
    # Padded 0's and made all the spectograms of equal length. Selected shape is [300 , 513]
    result_S_Val = np.zeros([300,513], dtype=np.float)
    T_result_S_Val= np.zeros([513,300], dtype=np.float)

    result_S_Val[:Inter_S_Val.shape[0], :Inter_S_Val.shape[1]] = Inter_S_Val
    T_result_S_Val[:S_Val.shape[0], :S_Val.shape[1]] = S_Val  
    
    S_cs_lst_Val.append(result_S_Val)
    S_Val_lst.append(T_result_S_Val)
    
S_cs_lst_Val = np.array(S_cs_lst_Val)
S_Val_lst = np.array(S_Val_lst)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/pytho

In [0]:

X_xs_lst_Val = []
T_X_Val=[]
vx_lst =[]

X_Val_lst=[]

for file in sorted(glob.iglob('/content/vx0*.wav')):
    vx, vx_sr=librosa.load(file, sr=None)
    X_Val=librosa.stft(vx, n_fft=1024, hop_length=512)
    
    vx_lst.append(vx)
    #print("X_Val.shape:", X_Val.shape)
    
    # Intermediate array which stores the Absolute and transposed spectogram data
    Inter_X_Val = np.transpose(np.abs(X_Val))
    #print(Inter_X.shape)


    
    # appending the actual length of signal
    T_X_Val.append(Inter_X_Val.shape[0])
    
    # Padded 0's and made all the spectograms of equal length. Selected shape is [300 , 513]
    result_X_Val = np.zeros([300,513], dtype=np.float)
    T_result_X_Val = np.zeros([513,300], dtype=np.float)

    result_X_Val[:Inter_X_Val.shape[0], :Inter_X_Val.shape[1]] = Inter_X_Val
    T_result_X_Val[:X_Val.shape[0], :X_Val.shape[1]] = X_Val   
    
    X_xs_lst_Val.append(result_X_Val)

    X_Val_lst.append(T_result_X_Val)

X_xs_lst_Val = np.array(X_xs_lst_Val)
X_Val_lst = np.array(X_Val_lst)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/pytho

In [0]:
# Shape for the validation data set

print(S_cs_lst_Val.shape)
print(X_xs_lst_Val.shape)

(100, 300, 513)
(100, 300, 513)


In [0]:
# complex arrays which i captured while reading the data in the above step.

print("X_Val_lst.shape", np.array(X_Val_lst).shape)
print("S_Val_lst.shape", np.array(S_Val_lst).shape)

X_Val_lst.shape (100, 513, 300)
S_Val_lst.shape (100, 513, 300)


In [0]:
# Ran the validation data against the Trained model and generated outputs.

X_hat_val = session.run(outputs_new, feed_dict={X_new: X_xs_lst_Val })

In [0]:
# outputs

X_hat_val.shape

(100, 300, 513)

In [0]:
# Checked the shape of Complex data (stft) and (output)

print(X_Val_lst.shape)
print(X_hat_val.shape)
print(X_hat_val.transpose(0,2,1).shape)

inpt = X_Val_lst
reslt = (X_hat_val.transpose(0,2,1))


(100, 513, 300)
(100, 300, 513)
(100, 513, 300)


In [0]:
# Multiply 

first_reslt = np.multiply(inpt,reslt)

In [0]:
print(first_reslt.shape)

(100, 513, 300)


In [0]:
# Calculated SNR, created file

SNR_lst=[]
for i in range(first_reslt.shape[0]):
 
  inp_wav = first_reslt[i]
  Clean_sound = vs_lst[i]
  
  name="V_out_"+str(i)+".wav"

  X_hat_recon_first = librosa.istft(inp_wav,  hop_length=512, win_length=1024) 
  
  librosa.output.write_wav(name, X_hat_recon_first, vx_sr)
  
  # As i had padded 0's and made the length 300. So, while calculating SNR i used the actual length and took only that may bytes from "X_hat_recon_first"
  
  SNR = 10*np.log10(np.sum(Clean_sound**2)/np.sum((Clean_sound - X_hat_recon_first[:(len(vs_lst[i]))])**2))
  SNR_lst.append(SNR)
  
  
  if (i % 10) == 0:
    
    print("i:", i,"inp_wav.shape:", inp_wav.shape, "name:", name,"(len(vs_lst[0])):", (len(vs_lst[i])), "SNR:", SNR)
  
  
print(np.mean(SNR_lst))

i: 0 inp_wav.shape: (513, 300) name: V_out_0.wav (len(vs_lst[0])): 33178 SNR: 0.5254633352160454
i: 10 inp_wav.shape: (513, 300) name: V_out_10.wav (len(vs_lst[0])): 66356 SNR: 0.5618662014603615
i: 20 inp_wav.shape: (513, 300) name: V_out_20.wav (len(vs_lst[0])): 37991 SNR: 0.6311026215553284
i: 30 inp_wav.shape: (513, 300) name: V_out_30.wav (len(vs_lst[0])): 34919 SNR: 0.5125168338418007
i: 40 inp_wav.shape: (513, 300) name: V_out_40.wav (len(vs_lst[0])): 46797 SNR: 0.5224847421050072
i: 50 inp_wav.shape: (513, 300) name: V_out_50.wav (len(vs_lst[0])): 52532 SNR: 0.6659930199384689
i: 60 inp_wav.shape: (513, 300) name: V_out_60.wav (len(vs_lst[0])): 44442 SNR: 0.8011444658041
i: 70 inp_wav.shape: (513, 300) name: V_out_70.wav (len(vs_lst[0])): 70656 SNR: 0.7552697509527206
i: 80 inp_wav.shape: (513, 300) name: V_out_80.wav (len(vs_lst[0])): 39834 SNR: 0.15963712707161903
i: 90 inp_wav.shape: (513, 300) name: V_out_90.wav (len(vs_lst[0])): 40551 SNR: 0.5967632308602333
0.559376303618

In [0]:

!ls V_out*

V_out_0.wav   V_out_14.wav  V_out_19.wav  V_out_4.wav  V_out_9.wav
V_out_10.wav  V_out_15.wav  V_out_1.wav   V_out_5.wav
V_out_11.wav  V_out_16.wav  V_out_20.wav  V_out_6.wav
V_out_12.wav  V_out_17.wav  V_out_2.wav   V_out_7.wav
V_out_13.wav  V_out_18.wav  V_out_3.wav   V_out_8.wav


In [0]:
# output file download

from google.colab import files

for i in range (first_reslt.shape[0]):
  name="V_out_"+str(i)+".wav"
  
  with open(name, 'w') as f:
    f.write(name)

    files.download(name)